In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/digipodium/Datasets/main/house_pricing.csv")
df.info()
df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814 entries, 0 to 813
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        814 non-null    object 
 1   Type        814 non-null    object 
 2   Beds        814 non-null    int64  
 3   Baths       814 non-null    int64  
 4   SquareFeet  814 non-null    int64  
 5   Price       814 non-null    float64
dtypes: float64(1), int64(3), object(2)
memory usage: 38.3+ KB


,City,Type,Beds,Baths,SquareFeet,Price
0,SACRAMENTO,Residential,2,1,836,138159.85
1,SACRAMENTO,Residential,3,1,1167,167541.46


In [3]:
# Feature selection (X, y) & Split into xtrain,xtest,ytrain,ytest
X = df[['Beds','Baths','SquareFeet']]
y = df['Price']
xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=.2, random_state=1)

In [4]:
print("Random Forest")
model2 = RandomForestRegressor()
model2.fit(xtrain,ytrain)
print("score:", model2.score(xtest,ytest) * 100)
pred = model2.predict(X)
print("mse:",mean_squared_error(y,pred))
print("mae:",mean_absolute_error(y,pred))

Random Forest
score: 71.62391918777087
mse: 866343368.9482411
mae: 15221.526921482024


In [5]:
forest_score = cross_val_score(model2,X,y,cv=6)
print(forest_score, f"average: {forest_score.mean():.2f}",f"std :{forest_score.std():.2f}")

[0.81862302 0.6964869  0.71435156 0.72747654 0.49508033 0.61765737] average: 0.68 std :0.10


#### Grid search

In [15]:
# we are going to create a dictionary with all the parameter and their value options
RandomForestRegressor?

Init signature:
RandomForestRegressor(
    n_estimators=100,
    *,
    criterion='squared_error',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
)
Docstring:     
A random forest regressor.

A random forest is a meta estimator that fits a number of classifying
decision trees on various sub-samples of the dataset and uses averaging
to improve the predictive accuracy and control over-fitting.
The sub-sample size is controlled with the `max_samples` parameter if
`bootstrap=True` (default), otherwise the whole dataset is used to build
each tree.

Read more in the :ref:`User Guide <forest>`.

Parameters
----------
n_estimators : int, default=100
    The number of trees in the forest.

    .. version

In [7]:
params = {
    'n_estimators' : list(range(100,501,100)),
    'criterion': ["squared_error", "absolute_error", "poisson"],
    'max_depth': list(range(5,51,15)),
}
params

{'n_estimators': [100, 200, 300, 400, 500],
 'criterion': ['squared_error', 'absolute_error', 'poisson'],
 'max_depth': [5, 20, 35, 50]}

In [8]:
grid = GridSearchCV(estimator=RandomForestRegressor(),param_grid=params,cv=3,n_jobs=-1,verbose=3)

In [9]:
grid.fit(X,y)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error',
                                       'poisson'],
                         'max_depth': [5, 20, 35, 50],
                         'n_estimators': [100, 200, 300, 400, 500]},
             verbose=3)

In [10]:
gf = pd.DataFrame(grid.cv_results_)

In [11]:
gf.sort_values(by='rank_test_score',inplace=True)
gf

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
22,1.321258,0.019851,0.025701,0.001837,absolute_error,5,300,"{'criterion': 'absolute_error', 'max_depth': 5...",0.824635,0.767305,0.615423,0.735787,0.088270,1
23,1.696276,0.032788,0.033379,0.004919,absolute_error,5,400,"{'criterion': 'absolute_error', 'max_depth': 5...",0.824244,0.768724,0.614224,0.735730,0.088857,2
24,2.099109,0.010706,0.047158,0.005667,absolute_error,5,500,"{'criterion': 'absolute_error', 'max_depth': 5...",0.824561,0.766977,0.614059,0.735199,0.088826,3
21,0.955360,0.026075,0.017185,0.000885,absolute_error,5,200,"{'criterion': 'absolute_error', 'max_depth': 5...",0.821285,0.766691,0.615682,0.734552,0.086959,4
20,0.545318,0.030475,0.009106,0.001156,absolute_error,5,100,"{'criterion': 'absolute_error', 'max_depth': 5...",0.821138,0.768917,0.611419,0.733824,0.089141,5
0,0.153435,0.003768,0.013328,0.003770,squared_error,5,100,"{'criterion': 'squared_error', 'max_depth': 5,...",0.794699,0.769200,0.625611,0.729837,0.074430,6
3,0.661513,0.032064,0.046600,0.004255,squared_error,5,400,"{'criterion': 'squared_error', 'max_depth': 5,...",0.800966,0.768995,0.613530,0.727830,0.081869,7
2,0.463291,0.013918,0.034786,0.004545,squared_error,5,300,"{'criterion': 'squared_error', 'max_depth': 5,...",0.802420,0.767168,0.613418,0.727669,0.082059,8
4,0.846812,0.027572,0.049507,0.003932,squared_error,5,500,"{'criterion': 'squared_error', 'max_depth': 5,...",0.801999,0.766816,0.613620,0.727478,0.081781,9
1,0.314399,0.012604,0.023476,0.002883,squared_error,5,200,"{'criterion': 'squared_error', 'max_depth': 5,...",0.797648,0.766320,0.612526,0.725498,0.080900,10


In [12]:
grid.best_estimator_

RandomForestRegressor(criterion='absolute_error', max_depth=5, n_estimators=300)

In [13]:
from joblib import dump

In [14]:
dump(grid.best_estimator_,"house_pricing_model_73.pkl")

['house_pricing_model_73.pkl']